# Assignment 1: Word2Vec Representations (10 Marks)

## Due: June 20, 2023

Welcome to the Assignment 1 of the course. This week we will learn about vector representations for words and how can we utilize them to solve the topic classification task that we discussed in the previous lab.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_dir = "gdrive/MyDrive/PlakshaNLP2023/Assignment1/data/ag_news_csv"
except:
    data_dir = "/datadrive/t-kabir/work/repos/PlakshaNLP/TLPNLP2023/source/Assignment1/data/ag_news_csv"

In [ ]:
# Install required libraries
!pip install numpy
!pip install pandas
!pip install nltk
!pip install torch
!pip install tqdm
!pip install matplotlib
!pip install seaborn
!pip install gensim

In [ ]:
# We start by importing libraries that we will be making use of in the assignment.
import string
import tqdm
import numpy as np
import pandas as pd
import torch
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

nltk.download("punkt")
nltk.download('stopwords')

Similar to last time we will again be working on the AG News Dataset. Below we load the dataset into the memory

In [ ]:
NUM_LABELS = 4
LABELS_MAP = ["World", "Sports", "Business", "Sci/Tech"]

def load_dataset(split):
    ## Load the datasets and specify the column names
    df = pd.read_csv(f"{data_dir}/{split}.csv", names=["label", "title", "description"])

    ## Merge the title and description columns
    df["news"] = df["title"] + " " + df["description"]

    ## Remove the title and description columns
    df = df.drop(["title", "description"], axis=1)

    ## Have the labels start from 0
    df["label"] = df["label"] - 1

    ## Map the label to the corresponding class
    df["label_readable"] = df["label"].apply(lambda x: LABELS_MAP[int(x)])

    df = df[["news", "label", "label_readable"]]

    # Shuffle the dataset
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    return df

## Load the datasets and specify the column names
train_df = load_dataset("train")
test_df = load_dataset("test")

print(f"Number of Training Examples: {len(train_df)}")
print(f"Number of Test Examples: {len(test_df)}")

In [ ]:
# View a sample of the dataset
train_df.head()

## Task 0: Warm Up Excercise (2 Marks)

To start we ask you to re-implement some functions from the Lab 1. Mainly you will implement the preprocessing pipeline and vocabulary building functions again as well as some new but related functions. Details about the functions will be given in their Doc Strings.

### Task 0.1: Preprocessing Pipeline (1 Mark)

Implement the preprocessing pipeline like we did in Lab1, however, this time we will only implement converting the text to lower case and removing punctuations.

We are not doing any stemming this time as we will be using pre-trained word representations in this assignment, and like it was discussed in the lectures stemming often results in the words that may not exist in common dictionaries.

We are also skipping stop words removal this time around, the reason being that removing stop words can often hurt the structural integrity of a sentence and the choice of stop words to use can be very subjective and depend upon the task at hand. For example: In the stop words list that we used last time contained the word `not`, removing which can change the sentiment of the sentence, eg. I did not like this movie -> I did like this movie. In this assignment we will explore more sophisticated ways to handle the stop words than just directly removing them from the text.

In [ ]:
def preprocess_pipeline(text):
    """
    Given a piece of text applies preprocessing techniques
    like converting to lower case, removing stop words and punctuations.

    Apply the functions in the following order:
    1. to_lower_case
    2. remove_punctuations

    Inputs:
    - text (str) : A python string containing text to be pre-processed

    Returns:
    - text_preprocessed (str) : Resulting string after applying preprocessing

    Note: You may implement the functions for the two steps seperately in this cell
            or just write all the code in this function only we leave that up to you.
    """

    text_preprocessed = None

    # YOUR CODE HERE
    raise NotImplementedError()

    return text_preprocessed

In [ ]:

def evaluate_string_test_cases(test_case_input,
                        test_case_func_output,
                        test_case_exp_output):

    print(f"Input: {test_case_input}")
    print(f"Function Output: {test_case_func_output}")
    print(f"Expected Output: {test_case_exp_output}")

    if test_case_func_output == test_case_exp_output:
        print("Test Case Passed :)")
        print("**********************************\n")
        return True
    else:
        print("Test Case Failed :(")
        print("**********************************\n")
        return False

print("Running Sample Test Cases")
print("Sample Test Case 1:")
test_case = "Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal!"
test_case_answer = "mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal"
test_case_student_answer = preprocess_pipeline(test_case)
assert evaluate_string_test_cases(test_case, test_case_student_answer, test_case_answer)

print("Sample Test Case 2:")
test_case = "\"Little tyke,\" chortled Mr. Dursley as He left the house."
test_case_answer = "little tyke chortled mr dursley as he left the house"
test_case_student_answer = preprocess_pipeline(test_case)
assert evaluate_string_test_cases(test_case, test_case_student_answer, test_case_answer)


In [ ]:
## Preprocess the dataset

train_df["news"] = train_df["news"].apply(lambda x : preprocess_pipeline(x))
test_df["news"] = test_df["news"].apply(lambda x : preprocess_pipeline(x))

### Task 0.2: Create Vocabulary (0.25 Marks)

Implement the `create_vocab` function below like you did during the lab. Do not forget using `nltk.tokenize.word_tokenize` to tokenize the text into words.

In [ ]:
def create_vocab(documents):
    """
    Given a list of documents each represented as a string,
    create a word vocabulary containing all the words that occur
    in these documents.
    (0.25 Marks)

    Inputs:
        - documents (list) : A list with each element as a string representing a
                            document.

    Returns:
        - vocab (list) : A **sorted** list containing all unique words in the
                        documents

    Example Input: ['john likes to watch movies mary likes movies too',
                  'mary also likes to watch football games']

    Expected Output: ['also',
                    'football',
                    'games',
                    'john',
                    'likes',
                    'mary',
                    'movies',
                    'to',
                    'too',
                    'watch']


    Hint: `nltk.tokenize.word_tokenize` function may come in handy

    """

    vocab = []

    # YOUR CODE HERE
    raise NotImplementedError()

    return sorted(vocab) # Don't change this

In [ ]:
def evaluate_list_test_cases(test_case_input,
                        test_case_func_output,
                        test_case_exp_output):

    print(f"Input: {test_case_input}")
    print(f"Function Output: {test_case_func_output}")
    print(f"Expected Output: {test_case_exp_output}")

    if test_case_func_output == test_case_exp_output:
        print("Test Case Passed :)")
        print("**********************************\n")
        return True
    else:
        print("Test Case Failed :(")
        print("**********************************\n")
        return False


print("Running Sample Test Cases")
print("Sample Test Case 1:")

test_case = ["john likes to watch movies mary likes movies too",
              "mary also likes to watch football games"]
test_case_answer = ['also', 'football', 'games', 'john', 'likes', 'mary', 'movies', 'to', 'too', 'watch']
test_case_student_answer = create_vocab(test_case)
assert evaluate_list_test_cases(test_case, test_case_student_answer, test_case_answer)

print("Sample Test Case 2:")

test_case = ["We all live in a yellow submarine.",
             "Yellow submarine, yellow submarine!!"
             ]
test_case_answer = ['!', ',', '.', 'We', 'Yellow', 'a', 'all', 'in', 'live', 'submarine', 'yellow']
test_case_student_answer = create_vocab(test_case)
assert evaluate_list_test_cases(test_case, test_case_student_answer, test_case_answer)



In [ ]:
# Create vocabulary from training data
train_documents = train_df["news"].values.tolist()
train_vocab = create_vocab(train_documents)

### Task 0.3: Get Word Frequencies (0.75 Marks)

We define the normalized frequency of a word `w` in a corpus as:

p(w) = Number of occurences of `w` in all documents / Total Number of occurences of all words in all documents

Word frequencies can be helpful as it can help us recognize the most common words which in most cases will be stop words as well as rare words that occur in the documents. Later we will be making use of word frequencies to create sentence representations, but for now just implement the `get_word_frequencies` below

In [ ]:
def get_word_frequencies(documents):
    """
    Gets the normalized frequency of each word w i.e.
    p(w) =  #num_of_occurences_of_w / #total_occurences_of_all_words
    present in documents

    Inputs:
        - documents(list): A list of documents

    Returns:
        - word2freq(dict): A dictionary containing words as keys
                           and values as their corresponding frequencies

    """

    word2freq = {}

    # YOUR CODE HERE
    raise NotImplementedError()

    return word2freq

In [ ]:
def check_dicts_same(dict1, dict2):
    if not isinstance(dict1, dict):
        print("Your function output is not a dictionary!")
        return False
    if len(dict1) != len(dict2):
        return False

    for key in dict1:
        val1 = dict1[key]
        val2 = dict2[key]
        if isinstance(val1, float) and isinstance(val1, float):
            if not np.allclose(val1, val2, 1e-4):
                return False
        if val1 != val2:
            return False

    return True

print("Running Sample Test Case 1")
sample_documents = [
    'john likes to watch movies mary likes movies too',
    'mary also likes to watch football games'
]
actual_word2freq = {'john': 0.0625,
                     'likes': 0.1875,
                     'to': 0.125,
                     'watch': 0.125,
                     'movies': 0.125,
                     'mary': 0.125,
                     'too': 0.0625,
                     'also': 0.0625,
                     'football': 0.0625,
                     'games': 0.0625}

output_word2freq = get_word_frequencies(sample_documents)
print(f"Input Documents: {sample_documents}")
print(f"Output Word Frequencies: {output_word2freq}")
print(f"Expected Word Frequencies: {actual_word2freq}")

assert check_dicts_same(output_word2freq, actual_word2freq)
print("****************************************\n")

print("Running Sample Test Case 2")
sample_documents = [
    'We all live in a yellow submarine.',
    'Yellow submarine, yellow submarine!!'
]
actual_word2freq = {'We': 0.06666666666666667,
                    'all': 0.06666666666666667,
                    'live': 0.06666666666666667,
                    'in': 0.06666666666666667,
                    'a': 0.06666666666666667,
                    'yellow': 0.13333333333333333,
                    'submarine': 0.2,
                    '.': 0.06666666666666667,
                    'Yellow': 0.06666666666666667,
                    ',': 0.06666666666666667,
                    '!': 0.13333333333333333}

output_word2freq = get_word_frequencies(sample_documents)
print(f"Input Documents: {sample_documents}")
print(f"Output Word Frequencies: {output_word2freq}")
print(f"Expected Word Frequencies: {actual_word2freq}")

assert check_dicts_same(output_word2freq, actual_word2freq)
print("****************************************\n")


## Task 1: Word2Vec Representations

In this task you will learn how to use word2vec for obtaining vector representations for words and then how to use them further to create sentence/document level vector representations. We will be using the popular [gensim](https://radimrehurek.com/gensim/) package that has great support for vector space models and supports various popular word embedding methods like word2vec, fasttext, LSA etc. For the purposes of this assignment we will be working with the pretrained word2vec vectors on the google news corpus containing about 100 billion tokens. Below we provide a tutorial on how to use gensim for obtaining these word vectors.

We start by downloading pretrained word2vec vectors and create a `gensim.models.keyedvectors` obect. The download has a size of about 2GB, so might take a few minutes to download and load.

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

The `wv` object has a bunch of methods that we can use to obtain vector representations of words, finding similar words etc. We start with how to obtain vectors for words using it, which can be done using the `get_vector` method as demonstrated below.

In [ ]:
word = "bad"
vector = wv.get_vector(word)
print(f"Word : {word}")
print(f"Length of the vector: {len(vector)}")
print(f"Vector:")
print(vector)

You can also obtain the brackets by using angular brackets notation i.e. `wv["bad"]`

In [ ]:
word = "bad"
vector = wv[word]
print(f"Word : {word}")
print(f"Length of the vector: {len(vector)}")
print(f"Vector:")
print(vector)

Also note that the word2vec model might not have vectors for all words, you can check for Out of Vocabulary (OOV) words using the `in` operator as shown in the code block below.

In [ ]:
print("book" in wv)
print("blastoise" in wv)

Just looking at the vectors we cannot really gain any insights about them, but it is the relation between the vectors of different words that is much more easier to interpet. `wv` object has a `most_similar` method that for a given word obtains the words that are most similar to it by computing cosine similarity between them.

In [ ]:
wv.most_similar("bad",topn=5)

In [ ]:
wv.most_similar("king",topn=5)

You can see that the we obtain very reasonable similar words in both examples. We can also use `most_similar` to do the analogy comparison that was discussed in the class. For eg: man : king :: woman : ?

In [ ]:
wv.most_similar(positive=['woman', 'king'], negative=['man'], topn = 1)

In [ ]:
wv.most_similar(positive=['woman', 'father'], negative=['man'], topn = 1)

### Task 1.1 Sentence representations using Word2Vec : Bag of Words Methods (2 Marks)

Now that we know how to obtain the vectors of each word, how can we obtain a vector representation for a sentence or a document? One of the simplest way is to add the vectors of all the words in the sentence to obtain sentence vector. This is also called the Bag of Words approach. Can you think of why? Last time when we discussed bag of words features for a sentence, it contained counts of each word occuring in the sentence. This can be just thought of as just adding one hot vectors for all the words in a sentence. Hence, adding word2vec vectors for each word in the sentence can also be viewed as a bag of words representation.

Implement the `get_bow_sent_vec` function below that takes in a sentence and adds the word2vec vectors for each word occuring in the sentence to obtain the sentence vector. Also, in practice it is helpful to divide the sum of word vectors by the number of words to normalize the representation obtained.

In [ ]:
def get_bow_sent_vec(sentence, wv):
    """
    Obtains the vector representation of a sentence by adding the word vectors
    for each word occuring in the sentence (and dividing by the number of words) i.e

    v(s) = sum_{w \in s}(v(w)) / N(s)
    where N(s) is the number of words in the sentence,
    v(w) is the word2vec representation for word w
    and v(s) is the obtained vector representation of sentence s

    Inputs:
        - sentence (str): A string containing the sentence to be encoded
        - wv (gensim.models.keyedvectors.KeyedVectors) : A gensim word vector model object.

    Returns:
        - sentence_vec (np.ndarray): A numpy array containing the vector representation
        of the sentence

    Note : Not all the words might be present in `wv` so you will need to check for that,
          and only add vectors for the words that are present. Also while normalization
          divide by the number of words for which a word vector was actually present in `wv`

    Important Note: In case no word in the sentence is present in `wv`, return an all zero vector!

    """

    sentence_vec = None

    # YOUR CODE HERE
    raise NotImplementedError()

    return sentence_vec

In [ ]:
print("Running Sample Test Case 1")
sample_sentence ='john likes watching movies mary likes movies too'
sentence_vec = get_bow_sent_vec(sample_sentence, wv)
expected_sent_vec = np.array([ 0.03330994,  0.11713409,  0.00738525,  0.24951172, -0.0202179 ])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")

print("Running Sample Test Case 2")
sample_sentence ='We all live in a yellow submarine.'
sentence_vec = get_bow_sent_vec(sample_sentence, wv)
expected_sent_vec = np.array([-0.08424886,  0.14601644,  0.0727946 ,  0.09978231, -0.02655029])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")

print("Running Sample Test Case 3")
sample_sentence ='blastoise pikachu charizard'
sentence_vec = get_bow_sent_vec(sample_sentence, wv)
expected_sent_vec = np.array([0.,  0.,  0. ,  0., 0.])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")


### Task 1.2 Sentence representations using Word2Vec : Inverse Frequency Weighted Sum Method (2 Marks)

Instead of directly adding the vectors for all the words in the sentence, we can do something slightly better which tends to work very well in practice. [Arora et al. 2017](https://openreview.net/pdf?id=SyK00v5xx) proposes the following method for computing sentence embedding from word vectors

<img src="https://i.ibb.co/vwzHXHy/sent-embed.jpg" alt="sent-embed" border="0">

Here v_w is the vector representation of the word w, p(w) is the frequency of the word w, |s| is the number of words in the sentence, and `a` is just a constant with a typical value between 1e-3 to 1e-4.

Intuitively, we take a weighted sum of all the word vectors where the weights are inversely propotional to the frequency of the word (p(w)). This ensures that very frequent words which are often stop words like "the", "I" etc. are given lower weightage when constructing the sentence vector. `a` is used as smoothing constant, such that when p(w) = 0 we still have finite weights.


In [ ]:
def get_weighted_bow_sent_vec(sentence, wv, word2freq, a = 1e-4):
    """
    Obtains the vector representation of a sentence by adding the word vectors
    for each word occuring in the sentence (and dividing by the number of words) i.e

    v(s) = (sum_{w \in s} a / (a + p(w)) * (v(w))) / N(s)

    Inputs:
        - sentence (str): A string containing the sentence to be encoded
        - wv (gensim.models.keyedvectors.KeyedVectors) : A gensim word vector model object.
        - word2freq (dict): A dictionary with words as keys and their frequency in the
                            entire training dataset as values
        - a (float): Smoothing constant

    Returns:
        - sentence_vec (np.ndarray): A numpy array containing the vector representation
        of the sentence

    Important Note: In case no word in the sentence is present in `wv`, return an all zero vector!

    Hint: If a word is not present in the `word2freq` dictionary, you can consider frequency
          of that word to be zero

    """

    sentence_vec = None

    # YOUR CODE HERE
    raise NotImplementedError()

    return sentence_vec

In [ ]:
print("Running Sample Test Case 1")
sample_sentence ='john likes watching movies mary likes movies too'
sample_word2freq = {
    "john" : 0.001,
    "likes": 0.01,
    "watching" : 0.01,
    "movies": 0.05,
    "mary" : 0.001,
    "too": 0.1
}
sentence_vec = get_weighted_bow_sent_vec(sample_sentence, wv, sample_word2freq)
expected_sent_vec = np.array([-0.00384654,  0.00208942,  0.00010824,  0.00648482, -0.00236967])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")

print("Running Sample Test Case 2")
sample_sentence ='We all live in a yellow submarine.'
sentence_vec = get_weighted_bow_sent_vec(sample_sentence, wv, word2freq = {}, a = 1e-3)
expected_sent_vec = np.array([-0.08424886,  0.14601644,  0.0727946 ,  0.09978231, -0.02655029])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")

print("Running Sample Test Case 3")
sample_sentence ='blastoise pikachu charizard'
sentence_vec = get_weighted_bow_sent_vec(sample_sentence, wv, word2freq = {}, a = 1e-3)
expected_sent_vec = np.array([0.,  0.,  0. ,  0., 0.])
print(f"Input Sentence: {sample_sentence}")
print(f"First five elements of output vector: {sentence_vec[:5]}")
print(f"Expected first five elements of output vector: {expected_sent_vec}")
assert np.allclose(sentence_vec[:5], expected_sent_vec, 1e-4)
print("Sample Test Case Passed")
print("*******************************\n")


Now that you have implemented the sentence vector functions, let's obtain sentence vectors for all the sentences in our training and test sets. This will take a few minutes

In [ ]:
train_documents = train_df["news"].values.tolist()
test_documents = test_df["news"].values.tolist()
train_vocab = create_vocab(train_documents)
train_word2freq = get_word_frequencies(train_documents)

train_bow_vectors = np.array([
    get_bow_sent_vec(document, wv)
    for document in train_documents
])
test_bow_vectors = np.array([
    get_bow_sent_vec(document, wv)
    for document in test_documents
])

train_w_bow_vectors = np.array([
    get_weighted_bow_sent_vec(document, wv, train_word2freq, a = 1e-3)
    for document in train_documents
])
test_w_bow_vectors = np.array([
    get_weighted_bow_sent_vec(document, wv, train_word2freq, a = 1e-3)
    for document in test_documents
])

## Task 2: Train a Topic Classifier using Sentence Vectors

This part will be just like Lab 1, but instead of the Bag of Word features we defined last time to train the classifier, we will use the sentence vectors obtained from word2vec.

### Define a Custom Dataset class

In [ ]:
from torch.utils.data import Dataset, DataLoader

class AGNewsDataset(Dataset):

    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


### Task 2.1: Define the Multinomial Logistic Regression Model (1 Mark)

Like last time define a Multinomial Logistic Regression model that takes as input the sentence vector and predicts the label.

In [ ]:
import torch
import torch.nn as nn

class MultinomialLogisticRegressionModel(nn.Module):

    def __init__(self, d_input, num_labels):
        """
        Define the architecture of a Multinomial Logistic Regression classifier.
        You will need to define two components, one will be the linear layer using
        nn.Linear, and a log-softmax activation function for the output
        (log-softmax is numerically more stable and as we will see later we just need to log of the probabilities to calculate the loss).

        Inputs:
          - d_input (int): The dimensionality or number of features in each input.
                            This will be required to define the linear layer
          - num_labels (int): The number of classes in the dataset.

        Hint: Recall that in multinomial logistic regression we obtain a `num_labels` probablilities (or log-probabilities in this case)
        value for each input that denotes how likely is the input belonging
        to each class.
        """
        #Need to call the constructor of the parent class
        super(MultinomialLogisticRegressionModel, self).__init__()

        self.linear_layer = None
        self.log_softmax_layer = None

        # YOUR CODE HERE
        raise NotImplementedError()


    def forward(self, x):
        """
        Passes the input `x` through the layers in the network and returns the output

        Inputs:
          - x (torch.tensor): A torch tensor of shape [batch_size, d_input] representing the batch of inputs

        Returns:
          - output (torch.tensor): A torch tensor of shape [batch_size,] obtained after passing the input to the network

        """
        output = None

        # YOUR CODE HERE
        raise NotImplementedError()

        return output

In [ ]:
print("Running Sample Test Cases")
torch.manual_seed(42)
d_input = 5
num_labels = 4
sample_lr_model = MultinomialLogisticRegressionModel(d_input = d_input,num_labels = num_labels)
print(f"Sample Test Case 1: Testing linear layer input and output sizes, for d_input = {d_input}")
in_features = sample_lr_model.linear_layer.in_features
out_features = sample_lr_model.linear_layer.out_features

print(f"Number of Input Features: {in_features}")
print(f"Number of Output Features: {out_features}")
print(f"Expected Number of Input Features: {d_input}")
print(f"Expected Number of Output Features: {4}")
assert in_features == d_input and out_features == 4

print("**********************************\n")
d_input = 24
num_labels=6
sample_lr_model = MultinomialLogisticRegressionModel(d_input = d_input,num_labels = num_labels)
print(f"Sample Test Case 2: Testing linear layer input and output sizes, for d_input = {d_input}")
in_features = sample_lr_model.linear_layer.in_features
out_features = sample_lr_model.linear_layer.out_features

print(f"Number of Input Features: {in_features}")
print(f"Number of Output Features: {out_features}")
print(f"Expected Number of Input Features: {d_input}")
print(f"Expected Number of Output Features: {6}")
assert in_features == d_input and out_features == 6
print("**********************************\n")

print(f"Sample Test Case 3: Checking if the model gives correct output")
test_input = torch.rand(d_input)
model_output = sample_lr_model(test_input)
model_output_np = model_output.detach().numpy()
expected_output = np.array([-1.2607676, -1.8947134, -2.0088696, -2.7715783, -2.0052252, -1.4487281])
print(f"Model Output: {model_output_np}")
print(f"Expected Output: {expected_output}")

assert np.allclose(model_output_np, expected_output, 1e-5)
print("**********************************\n")

print(f"Sample Test Case 4: Checking if the model gives correct output")
test_input = torch.rand(4, d_input)
model_output = sample_lr_model(test_input)
model_output_np = model_output.detach().numpy()
expected_output = np.array([-1.4812257, -1.9529424, -1.8019284, -2.575539,  -2.2114434, -1.272432 ])
print(f"Model Output: {model_output_np}")
print(f"Expected Output: {expected_output}")

assert model_output_np[0].shape == expected_output.shape and np.allclose(model_output_np[0], expected_output, 1e-5)
print("**********************************\n")


### Task 2.2: Training and Evaluating the Model (3 Marks)

Write the training and evaluation script like the last time to train and evaluate topic classification model. You will need to write the entire functions on your own this time. You can refer to the code in Lab 1.

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam

def train(model, train_dataloader,
          lr = 1e-3, num_epochs = 20,
          device = "cpu",):

    """
    Runs the training loop

    Inputs:
    - model (MultinomialLogisticRegressionModel): Multinomial Logistic Regression model to be trained
    - train_dataloader (torch.utils.DataLoader): A dataloader defined over the training dataset
    - lr (float): The learning rate for the optimizer
    - num_epochs (int): Number of epochs to train the model for.
    - device (str): Device to train the model on. Can be either 'cuda' (for using gpu) or 'cpu'

    Returns:
    - model (MultinomialLogisticRegressionModel): Model after completing the training
    - epoch_loss (float) : Loss value corresponding to the final epoch
    """

    # YOUR CODE HERE
    raise NotImplementedError()
    return model, epoch_loss

In [ ]:
torch.manual_seed(42)
print("Training on 100 data points for sanity check")
sample_documents = train_df["news"].values.tolist()[:100]
sample_labels = train_df["label"].values.tolist()[:100]
sample_features = np.array([get_bow_sent_vec(document, wv) for document in sample_documents])
sample_dataset = AGNewsDataset(sample_features, sample_labels)
sample_dataloader = DataLoader(sample_dataset, batch_size=64)
sample_lr_model = MultinomialLogisticRegressionModel(d_input = len(sample_features[0]),num_labels=4)

sample_lr_model, loss = train(sample_lr_model, sample_dataloader,
      lr = 1e-2, num_epochs = 10,
      device = "cpu")

expected_loss = 0.9724720418453217
print(f"Final Loss Value: {loss}")
print(f"Expected Loss Value: {expected_loss}")

Don't worry if the loss values do not match exactly but you should see a decreasing trend and the final value should be of the same order of magnitude

In [ ]:
def evaluate(model, test_dataloader, device = "cpu"):
    """
    Evaluates `model` on test dataset

    Inputs:
    - model (MultinomialLogisticRegressionModel): Logistic Regression model to be evaluated
    - test_dataloader (torch.utils.DataLoader): A dataloader defined over the test dataset

    Returns:
    - accuracy (float): Average accuracy over the test dataset
    - preds (np.ndarray): Predictions of the model on test dataset
    """
    model.to(device)
    model = model.eval() # Set model to evaluation model
    accuracy = 0
    preds = []
    # YOUR CODE HERE
    raise NotImplementedError()
    return accuracy

In [ ]:
print(f"Testing the sample model on 100 examples for sanity check")
torch.manual_seed(42)
sample_documents = test_df["news"].values.tolist()[:100]
sample_labels = test_df["label"].values.tolist()[:100]
sample_features = np.array([get_bow_sent_vec(document, wv) for document in sample_documents])

sample_dataset = AGNewsDataset(sample_features,
                            sample_labels)

sample_dataloader = DataLoader(sample_dataset, batch_size = 64)
accuracy = evaluate(sample_lr_model, sample_dataloader, device ="cpu")
expected_accuracy = 0.7161458333333333
print(f"Accuracy: {accuracy}")
print(f"Expected Accuracy: {expected_accuracy}")

Now that you have implemented the training and evaluation functions, we will train (and evaluate) 2 different models and compare their performance. The 2 models are:

    - Multinomial Logistic Regression with Bag of Word2vec features
    - Multinomial Logistic Regression with Weighted Bag of Word2vec features

In [ ]:
print(f"Training and Evaluating Multinomial Logistic Regression with Bag of Word2vec features")
device = "cuda" if torch.cuda.is_available() else "cpu"

train_labels = train_df["label"].values.tolist()
test_labels = test_df["label"].values.tolist()

train_dataset = AGNewsDataset(train_bow_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size = 64)

test_dataset = AGNewsDataset(test_bow_vectors, test_labels)
test_loader = DataLoader(test_dataset, batch_size = 64)

lr_bow_model = MultinomialLogisticRegressionModel(
    d_input = wv.vector_size,num_labels= 4
)

lr_bow_model, loss = train(lr_bow_model, train_loader,
      lr = 1e-2, num_epochs = 10,
      device = device)

test_accuracy = evaluate(
    lr_bow_model, test_loader,
    device = device
)

print(f"Test Accuracy: {test_accuracy}")

In [ ]:
print(f"Training and Evaluating Multinomial Logistic Regression with Weighted Bag of Word2vec features")
device = "cuda" if torch.cuda.is_available() else "cpu"

train_labels = train_df["label"].values.tolist()
test_labels = test_df["label"].values.tolist()

train_dataset = AGNewsDataset(train_w_bow_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size = 64)

test_dataset = AGNewsDataset(test_w_bow_vectors, test_labels)
test_loader = DataLoader(test_dataset, batch_size = 64)

lr_bow_model = MultinomialLogisticRegressionModel(
    d_input = wv.vector_size, num_labels=4
)

lr_bow_model, loss = train(lr_bow_model, train_loader,
      lr = 1e-2, num_epochs = 10,
      device = device)

test_accuracy = evaluate(
    lr_bow_model, test_loader,
    device = device
)

print(f"Test Accuracy: {test_accuracy}")

First thing that you can notice is that these models train substantially faster than the models in Lab 1, as now we have much more lower sized sentence representations i.e. 300, compared to last time when it was equal to the size of vocabulary i.e. around 10k!

Both models get around ~88% test accuracy, which is close to what we got with Bag of Words features in Lab 1 only. The reason we do not see much improvement in performance is because both models still take a (weighted) sum of the individual word vectors to obtain sentence vectors, and fails to encode any structural information as well as semantics properly. For eg. for sentiment analysis task, both of the following sentences:

- it was a good movie adapted from a bad book
- it was a bad movie adapted from a good book

both of these sentences will get exact similar vector representations according to both the methods and hence the model will never be able to distinguish between the sentiment of these two sentences giving same prediction for both.

In the next labs and assignments we shall see how we can learn more contextual representation of the sentences that can help us solve the task much more efficiently.
